## Fork of the work done by EDA Team 1

In [28]:
class cfg():
    train_path = "/Users/user/pythonpractice/Omdena/Predicting Student Sucess Omdena Ankara 2023/train-student-mat.csv"
    test_path = "/Users/user/pythonpractice/Omdena/Predicting Student Sucess Omdena Ankara 2023/test-student-mat.csv"

    test_size = 0.3

    loss = "mean_absolute_error"

In [29]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway

In [30]:
# Function created by the EDA Team 1

def get_ses_status_score(row):
    # Calculate the SES score based on relevant features
    ses_score = 0

    # Add points based on parents' education
    if row['Medu'] in ['0']:
        ses_score += 1
    elif row['Medu'] in ['1', '2']:
        ses_score += 2
    elif row['Medu'] == '3':
        ses_score += 3
    elif row['Medu'] == '4':
        ses_score += 4

    if row['Fedu'] in ['0', '1']:
        ses_score += 1
    elif row['Fedu'] == '2':
        ses_score += 2
    elif row['Fedu'] == '3':
        ses_score += 3
    elif row['Fedu'] == '4':
        ses_score += 4

    # Add points based on parents' occupation (Mother_job and Father_job)
    if row['Mjob'] in ['at_home', 'other']:
        ses_score += 1
    elif row['Mjob'] in ['teacher', 'health', 'services']:
        ses_score += 2

    if row['Fjob'] in ['at_home', 'other']:
        ses_score += 1
    elif row['Fjob'] in ['teacher', 'health', 'services']:
        ses_score += 2

    # Add points based on family size (famsize)
    if row['famsize'] == 'LE3':
        ses_score += 1
        
    # Add points based on address (Urban or Rural)
    if row['address'] == 'U':
        ses_score += 1

    # Add points based on Parents_status (Together or Apart)
    if row['Pstatus'] == 'T':
        ses_score += 1
        
    # Add points based on guardian
    if row['guardian'] in ['mother', 'father']:
        ses_score += 1
    elif row['guardian'] == 'other':
        ses_score += 2

    # Add points based on school_support, fam_support, paid_extra_classes, higher_edu, internet, and nursery
    if row['schoolsup'] == 'yes':
        ses_score += 1

    if row['famsup'] == 'yes':
        ses_score += 1

    if row['paid'] == 'yes':
        ses_score += 1

    if row['higher'] == 'yes':
        ses_score += 1

    if row['internet'] == 'yes':
        ses_score += 1

    if row['nursery'] == 'yes':
        ses_score += 1

    return ses_score

In [31]:
# Reading dataframes

train = pd.read_csv(cfg.train_path)
test = pd.read_csv(cfg.test_path)

In [32]:
# Code block to calculate ses score for both train and test dataframes 
# FORK from EDA Team 1's work

train['ses_score'] = train.apply(get_ses_status_score, axis=1)
test['ses_score'] = test.apply(get_ses_status_score, axis=1)


# Calculate quartiles to derive threshold values
q1 = np.percentile(train['ses_score'], 25)
q2 = np.percentile(train['ses_score'], 50)  # Median
q3 = np.percentile(train['ses_score'], 75)

# Print the calculated quartiles
print("1st Quartile (25th percentile):", q1)
print("2nd Quartile (Median, 50th percentile):", q2)
print("3rd Quartile (75th percentile):", q3)

def get_ses_status_category(row):
    # Categorize SES score into 'Low', 'Medium', and 'High'
    if row['ses_score'] < q1:
        return 'Low'
    elif row['ses_score'] <= q2:
        return 'Medium'
    else:
        return 'High'
    
train['ses_status'] = train.apply(get_ses_status_category, axis=1)
test['ses_status'] = test.apply(get_ses_status_category, axis=1)

# Dropping the features used in calculating ses_status
train = train.drop(['address', 'famsize' , 'Pstatus','Medu','Fedu','Mjob','Fjob',
            'guardian','famsup','schoolsup', 'internet','paid','higher','nursery', 'ses_score', 'Unnamed: 0'], axis = 1)
test = test.drop(['address', 'famsize' , 'Pstatus','Medu','Fedu','Mjob','Fjob',
            'guardian','famsup','schoolsup', 'internet','paid','higher','nursery', 'ses_score', 'Unnamed: 0'], axis = 1)






1st Quartile (25th percentile): 9.0
2nd Quartile (Median, 50th percentile): 10.0
3rd Quartile (75th percentile): 11.0


In [33]:
train['avg_alc'] = (train['Dalc'] + train['Walc']) / 2
train = train.drop(['Dalc', 'Walc'], axis = 1)

test['avg_alc'] = (test['Dalc'] + test['Walc']) / 2
test = test.drop(['Dalc', 'Walc'], axis = 1)

In [34]:
label = {'no': 0, 'yes': 1}
train['romantic'] = train['romantic'].map(label)
test['romantic'] = test['romantic'].map(label)

# Perform the ANOVA test for go out with friends vs. Final Score
goout_anova_result = f_oneway(
    train[train['goout'] == 1]['Final_Score'],
    train[train['goout'] == 2]['Final_Score'],
    train[train['goout'] == 3]['Final_Score'],
    train[train['goout'] == 4]['Final_Score'],
    train[train['goout'] == 5]['Final_Score']
)

# Perform the ANOVA test for family relationship vs. Final Score
fam_anova_result = f_oneway(
    train[train['famrel'] == 1]['Final_Score'],
    train[train['famrel'] == 2]['Final_Score'],
    train[train['famrel'] == 3]['Final_Score'],
    train[train['famrel'] == 4]['Final_Score'],
    train[train['famrel'] == 5]['Final_Score']
)

# Perform the ANOVA test for Romantic relationship vs. Final Score
rom_anova_result = f_oneway(
    train[train['romantic'] == 1]['Final_Score'],
    train[train['romantic'] == 0]['Final_Score'] 
)

print("ANOVA test result:")

# Print the ANOVA test result for go out with friends vs final score
print("go out with friend vs. Final Score:  F-statistic =", goout_anova_result.statistic, " p =", goout_anova_result.pvalue)

# Print the ANOVA test result for family relationship vs final score
print("family relationship vs. Final Score: F-statistic =", fam_anova_result.statistic, "p =", fam_anova_result.pvalue)

# Print the ANOVA test result for relationship vs final score
print("romantic relationship vs. Final Score:F-statistic =", rom_anova_result.statistic, "  p =", rom_anova_result.pvalue)

ANOVA test result:
go out with friend vs. Final Score:  F-statistic = 3.016701015389935  p = 0.01850246265483004
family relationship vs. Final Score: F-statistic = 0.6095286286185053 p = 0.6561039147721099
romantic relationship vs. Final Score:F-statistic = 2.363818079632197   p = 0.12532769637755245


In [35]:
def calculate_social_score(row):
    friends_weight = 0.55
    romantic_weight = 0.35
    family_weight = 0.10

    # Calculate the social score using weighted average
    social_score = (row['famrel'] * family_weight +
                    row['goout'] * friends_weight +
                    row['romantic'] * romantic_weight)

    # Round the social_score to the nearest integer and return it
    return int(round(social_score))

In [36]:
# Apply the function to create the 'social_score' feature in the DataFrame
train['social_score'] = train.apply(calculate_social_score, axis=1)
test['social_score'] = test.apply(calculate_social_score, axis=1)


q1 = np.percentile(train['social_score'], 25)
q2 = np.percentile(train['social_score'], 50)  # Median
q3 = np.percentile(train['social_score'], 75)

# Print the calculated quartiles
print("1st Quartile (25th percentile):", q1)
print("2nd Quartile (Median, 50th percentile):", q2)
print("3rd Quartile (75th percentile):", q3)


# Define the categories for 'social_score'
train['social_score_category'] = pd.cut(train['social_score'], bins=[0, q1, q3, 1000], labels=['Very Bad', 'Neutral', 'Excellent'])
train = train.drop(['famrel', 'romantic', 'goout', 'social_score'], axis = 1)

test['social_score_category'] = pd.cut(test['social_score'], bins=[0, q1, q3, 1000], labels=['Very Bad', 'Neutral', 'Excellent'])
test = test.drop(['famrel', 'romantic', 'goout', 'social_score'], axis = 1)


1st Quartile (25th percentile): 2.0
2nd Quartile (Median, 50th percentile): 2.0
3rd Quartile (75th percentile): 3.0
